In [1]:
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install geopy

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Get the postal code to find out the cities information

In [40]:
html_data=requests.get('https://web.archive.org/web/20120613185331/http://www.postur.is/en/desktopdefault.aspx/tabid-501/681_read-1633/').text

In [41]:
soup=BeautifulSoup(html_data, 'lxml')
#print(soup.title)
soup

<!DOCTYPE HTML>
<html lang="en">
<head><script src="//archive.org/includes/analytics.js?v=cf34f82" type="text/javascript"></script>
<script type="text/javascript">window.addEventListener('DOMContentLoaded',function(){var v=archive_analytics.values;v.service='wb';v.server_name='wwwb-app28.us.archive.org';v.server_ms=286;archive_analytics.send_pageview({});});</script><script charset="utf-8" src="/_static/js/playback.bundle.js?v=bQvHU8mx" type="text/javascript"></script>
<script charset="utf-8" src="/_static/js/wombat.js?v=cRqOKCOw" type="text/javascript"></script>
<script type="text/javascript">
  __wm.init("https://web.archive.org/web");
  __wm.wombat("http://www.postur.is:80/en/desktopdefault.aspx/tabid-501/681_read-1633/","20120613185331","https://web.archive.org/","web","/_static/",
	      "1339613611");
</script>
<link href="/_static/css/banner-styles.css?v=wiOP2tld" rel="stylesheet" type="text/css"/>
<link href="/_static/css/iconochive.css?v=qtvMKcIJ" rel="stylesheet" type="text/c

In [63]:
column_names = ['Postal Code', 'City', 'Neighborhood', 'Closing Time', 'Home Delivery', 'Business Delivery', 'Additional Services']
df = pd.DataFrame(columns=column_names)
df

,Postal Code,City,Neighborhood,Closing Time,Home Delivery,Business Delivery,Additional Services


In [68]:
table = soup.find('table')
row_num = 0
for row in table.find("tbody").find_all("tr"):
    
    #print(row)
    col = row.find_all("td")
    #print(col)
    if (col != []):
        postalcode = col[0].text
        
        city = col[1].text
        neighborhood = col[2].text
        closetime = col[3].text
        homedelivery = col[4].text
        businessdelivery = col[5].text
        additionalservice = col[6].text
    
    df = df.append({'Postal Code': postalcode,
                'City':city,
                'Neighborhood': neighborhood,
                'Closing Time': closetime,
                'Home Delivery': homedelivery,
                'Business Delivery': businessdelivery,
                'Additional Services': additionalservice}, ignore_index=True)
        
df.head()


,Postal Code,City,Neighborhood,Additional Services,Business Delivery,Closing Time,Home Delivery
0,902,Vestmannaeyjum,Vestmannabraut 22 (pósthólf),,,,
1,101,Reykjavík,Pósthússtræti 5,available,09:00-17:00,16:30,17:00-22:00
2,102,(Millilanda Póstur),"Póstmiðstöð, Stórhöfða 32",,,,
3,103,Reykjavík,Síðumúla 3-5,available,09:00-17:00,16:30,17:00-22:00
4,104,Reykjavík,Síðumúla 3-5,available,09:00-17:00,16:30,17:00-22:00


In [69]:
df.shape

(150, 7)

In [70]:
df=df.drop(['Closing Time', 'Home Delivery', 'Business Delivery', 'Additional Services'], axis=1)


## Postal code associated with the city, the more count indicates it is a large city 

In [71]:
print('Iceland has {} unique cities'.format(
       len(df['City'].unique()))
    )

cityL=df['City'].value_counts() # count the number of postal code each city has
print('The largest city in Iceland with most postal code is ', 
      cityL.head(1))
 

Iceland has 87 unique cities
The largest city in Iceland with most postal code is  Reykjavík    23
Name: City, dtype: int64


In [72]:
# the five largest cities in Iceland
cityL.head()

Reykjavík      23
Reykjanesbæ     5
Kópavogi        4
Akureyri        4
Mosfellsbæ      3
Name: City, dtype: int64

## Find the neighborhoods in Reykjavik

In [73]:
df.head()

,Postal Code,City,Neighborhood
0,902,Vestmannaeyjum,Vestmannabraut 22 (pósthólf)
1,101,Reykjavík,Pósthússtræti 5
2,102,(Millilanda Póstur),"Póstmiðstöð, Stórhöfða 32"
3,103,Reykjavík,Síðumúla 3-5
4,104,Reykjavík,Síðumúla 3-5


In [75]:
## only display Reykjavik in City column
df=df.loc[df['City']=='Reykjavik']
df.head()


,Postal Code,City,Neighborhood


## Remove duplicate rows where City and Neighborhood value are the same 

In [76]:
## Remove duplicate rows when City and Neighborhood pairs are the same 
df.drop_duplicates(df['City'],df['Neighborhood'])

,Postal Code,City,Neighborhood


In [ ]:
column_name = ('City', 'District', 'Latitude', 'Longitude')
ice_df = pd.DataFrame(columns=column_names)

ice_df['City']='Reykjavik'
ice_df['District']=district

ice_df=ice_df.append({'City':'Reykjavik',
                      'District':district})
ice_df.head


## Get the Latitude and Longitude of Reykjavik

In [ ]:
address = 'Reykjavik, Iceland'

geolocator = Nominatim(user_agent="Iceland_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Iceland are {}, {}.'.format(latitude, longitude))

## Create a map of Reykjavik

In [ ]:
# create map of Reykjavik using latitude and longitude values
map_reykjavik = folium.Map(location=[latitude, longitude], zoom_start=11, tiles='Stamen Terrain')
map_reykjavik

## Explore Reykjavik using Foursquare API

In [ ]:
CLIENT_ID = 'IPFKGSTEYOILEGG2MJRFNKDX2YV5WEH44MR2KZDKZOWFBXWM' # your Foursquare ID
CLIENT_SECRET = '21MX2NLSSB4PWVFR3Y04NYJJOQCNDEWTM0TMVVLVKXK2B3CD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
# create map of Reykjavik using latitude and longitude values
map_reykjavik = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(reykjavik_data['Latitude'], reykjavik_data['Longitude']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_reykjavik)  
    
map_reykjavik

## Create a Iceland Dataframe to explore

In [ ]:
column_names = ['City', 'Latitude', 'Longitude']
ice_df = pd.DataFrame(columns=column_names)
ice_df['City']= df['City'].unique()
ice_df.head()

## Get the Latitude and Longitude of Reykjavik

In [ ]:
for city in ice_df['City']

    geolocator = Nominatim(user_agent="Iceland_explorer")
    location = geolocator.geocode(city)
    latitude = location.latitude
    longitude = location.longitude
    
    ice_df['City']=city
    ice_df['Latitude']=latitude
    ice_df['Longitude']=longitude

    ice_df=ice_df.append({'City': city,
                          'Latitude'=latitude,
                          'Longtitude'=longitude}, ignore_index=True)

ice_df

In [ ]:
address = 'Reykjavik, Iceland'

geolocator = Nominatim(user_agent="Iceland_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Iceland are {}, {}.'.format(latitude, longitude))

In [ ]:
address = 'Reykjanesbæ, Iceland'

geolocator = Nominatim(user_agent="Iceland_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Iceland are {}, {}.'.format(latitude, longitude))

## Create a Iceland Dataframe to explore

In [ ]:
column_names = ['City', 'Latitude', 'Longitude']
ice_df = pd.DataFrame(columns=column_names)
ice_df['City']=cityL
ice_df['Latitude']=latitude
ice_df['Longitude']=longitude
ice_df